# Fix Sklearn Version

This notebook ensures the AI-Lab using the same version of python library `scikit-learn` as the one used by the <a href="https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm" target="_blank" rel="noopener">Script Language Container (SLC)</a> inside Exasol database.

## Rationale

Using identical versions is required when transferring the sklearn model from the AI-Lab to the database SLC.

The AI-Lab serializes the sklearn model and uploads it into the database SLC. The SLC can only _deserialize_ the model if using the same version of the `scikit-learn` library. The specific version of the library used by the SLC depends on the release version of the database and cannot be controlled by the AI-Lab. Claiming to be intelligent, the AI-Lab adapts its own version of the library if required.

## Detect the Version of Library `scikit-learn` Inside the SLC and Replace Installed Version if Required

In [5]:
#--/
#CREATE OR REPLACE PYTHON3 SCALAR SCRIPT detect_skikit_learn_version() RETURNS VARCHAR(100) AS
#import sklearn
#def run(ctx):
#    return sklearn.__version__ 
#/

#select detect_skikit_learn_version();

# python -c "import sklearn; print(sklearn.__version__)"
# %pip install "scikit_learn==1.4.2"

Now we are ready to use this model in our SQL queries. This will be demonstrated in the [following notebook](sklearn_predict_abalone.ipynb).